In [69]:
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
from collections import Counter

In [2]:
n_samples = 2000
n_features = 1000
n_components = 10
n_top_words = 20

In [3]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [6]:
print("Loading dataset...")
t0 = time()
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))
data_samples = dataset.data[:n_samples]
print("done in %0.3fs." % (time() - t0))

Loading dataset...
done in 28.772s.


In [13]:
dataset.data[2]

"Although I realize that principle is not one of your strongest\npoints, I would still like to know why do do not ask any question\nof this sort about the Arab countries.\n\n   If you want to continue this think tank charade of yours, your\nfixation on Israel must stop.  You might have to start asking the\nsame sort of questions of Arab countries as well.  You realize it\nwould not work, as the Arab countries' treatment of Jews over the\nlast several decades is so bad that your fixation on Israel would\nbegin to look like the biased attack that it is.\n\n   Everyone in this group recognizes that your stupid 'Center for\nPolicy Research' is nothing more than a fancy name for some bigot\nwho hates Israel."

In [18]:
data_samples=[]
for line in open('./wechat.txt'):
    data_samples.append(line.strip())

In [19]:
# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features)
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()

Extracting tf features for LDA...
done in 86.025s.



In [95]:
tf_feature_names = tf_vectorizer.get_feature_names()
tf_feature_names=dict(zip(tf_feature_names,range(len(tf_feature_names))))
fout1=open('./transfered_data_text.txt','w')
fout2=open('./transfered_data_id.txt','w')
fout3=open('./transfered_data_libsvm.txt','w')
fout4=open('./feature_names.txt','w')
fout4.write('\n'.join(tf_feature_names.keys()))
for i,d in enumerate(data_samples):
    d1=list(filter(lambda w:w in tf_feature_names,d.split(' ')))
    fout1.write(' '.join(d1)+'\n')
    d2=list(map(lambda w:str(tf_feature_names[w]),d1))
    fout2.write(' '.join(d2)+'\n')
    d3=Counter(d2)
    d3=list(map(lambda v:'%s:%d'%(v),d3.items()))
    fout3.write(' '.join(d3)+'\n')


In [20]:
print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

Fitting LDA models with tf features, n_samples=2000 and n_features=1000...
done in 181.309s.


In [21]:
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topics in LDA model:
Topic #0: 健康 身体 作用 分钟 疾病 皮肤 治疗 患者 食物 时间 人体 医生 宝宝 效果 方法 营养 运动 功能 导致 功效
Topic #1: 工作 公司 学习 时间 管理 员工 能力 客户 企业 团队 经验 课程 老板 专业 成功 老师 销售 方法 方式 事情
Topic #2: 关注 二维码 识别 女人 免费 简介 微信 订阅 男人 点击 生活 健康 分享 选择 添加 id 轻松 技巧 搭配 公众
Topic #3: 产品 用户 技术 数据 公司 平台 互联网 手机 行业 市场 品牌 企业 服务 中国 发展 系统 智能 科技 网络 模式
Topic #4: 微信 点击 阅读 浏览器 播放 回复 原文 音乐 支持 文章 朋友 查看 视频 公众 内容 语音 时间 关注 手机 下方
Topic #5: 公司 市场 投资 汽车 银行 基金 资金 风险 上市 金融 价格 车型 股票 股份 资产 交易 业务 机构 行业 证券
Topic #6: 设计 时间 生活 电影 搭配 世界 活动 空间 选择 喜欢 中国 品牌 第一 酒店 风格 推荐 地址 时尚 感觉 自然
Topic #7: 孩子 生活 女人 男人 喜欢 父母 朋友 妈妈 人生 时间 事情 家庭 东西 发现 世界 幸福 工作 儿子 第一 感觉
Topic #8: 中国 美国 经济 国家 城市 投资 发展 市场 企业 韩国 北京 政府 政策 房地产 房价 金融 日本 公司 乐天 社会
Topic #9: 工作 服务 建设 管理 人员 项目 发展 国家 信息 全国 交通 单位 企业 工程 中心 相关 有限公司 招聘 标准 部门

